In [ ]:
# 다양한 csv 파일 다루기
import pandas as pd

data = pd.read_csv('./train.csv')
print(data)

print(data.isnull().sum())

# 빈칸에 평균값 넣기

평균 = data['Age'].mean()
print(평균)
최빈값 = data['Embarked'].mode()
print(최빈값)

# 빈칸채우기
data['Age'].fillna(value = 30,inplace=True) # 이때 30은 출력을 해보고 내가 직접 넣은 결과 

# Embarked최빈값넣기

data['Embarked'].fillna(value = 'S', inplace=True) # S 역시 같다.

print(data.isnull().sum())






     PassengerId  Survived  Pclass  ...            Ticket     Fare  Embarked
0              1         0       3  ...         A/5 21171   7.2500         S
1              2         1       1  ...          PC 17599  71.2833         C
2              3         1       3  ...  STON/O2. 3101282   7.9250         S
3              4         1       1  ...            113803  53.1000         S
4              5         0       3  ...            373450   8.0500         S
..           ...       ...     ...  ...               ...      ...       ...
886          887         0       2  ...            211536  13.0000         S
887          888         1       1  ...            112053  30.0000         S
888          889         0       3  ...        W./C. 6607  23.4500         S
889          890         1       1  ...            111369  30.0000         C
890          891         0       3  ...            370376   7.7500         Q

[891 rows x 11 columns]
PassengerId      0
Survived         0
Pclass       

In [ ]:
import tensorflow as tf

정답 = data.pop('Survived')

ds = tf.data.Dataset.from_tensor_slices(( dict(data), 정답 ))
print(ds)
# feature colum 이용할려면 이런데이터 셋이 나와야함

<TensorSliceDataset shapes: ({PassengerId: (), Pclass: (), Name: (), Sex: (), Age: (), SibSp: (), Parch: (), Ticket: (), Fare: (), Embarked: ()}, ()), types: ({PassengerId: tf.int64, Pclass: tf.int64, Name: tf.string, Sex: tf.string, Age: tf.float64, SibSp: tf.int64, Parch: tf.int64, Ticket: tf.string, Fare: tf.float64, Embarked: tf.string}, tf.int64)>


In [ ]:
feature_columns = [] # 각열을 어떻게 바꿔서 넣을 건지 여기서 정의
#숫자로 집어넣을거 : numeric_column
#뭉퉁그려서 집어넣을거 : bucketized_column
#종류몇개없는 카테고리화해서 집어넣을거 : indicator_column
#종류가 넘많은 카테고리 : embedding_column

# 참고 :노말라이저 함수 추가가능
def 노말라이저함수(x) :
  
  최소 = data['Fare'].min()
  최대 = data['Fare'].max()
  
  return (x-최소)/ (최대 - 최소)


feature_columns.append(tf.feature_column.numeric_column('Fare', normalizer_fn=노말라이저함수))
feature_columns.append(tf.feature_column.numeric_column('Parch'))
feature_columns.append(tf.feature_column.numeric_column('SibSp'))
# feature_columns.append(tf.feature_column.numeric_column('Age'))


Age = tf.feature_column.numeric_column('Age')
Age_bucket=tf.feature_column.bucketized_column(Age, boundaries=[10,20,30,40,50,60])
feature_columns.append(Age_bucket)

NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [ ]:
one_list = ['Sex','Embarked','Pclass']

for i in one_list :
  vocab = data[i].unique() #유니크 문자열 바로만들기 pandas
  # print(vocab)
  cat = tf.feature_column.categorical_column_with_vocabulary_list(i,vocab)
  one_hot = tf.feature_column.indicator_column(cat)
  feature_columns.append( one_hot )

# embedding

vocab = data['Ticket'].unique() #유니크 문자열 바로만들기 pandas
# print(vocab)
cat = tf.feature_column.categorical_column_with_vocabulary_list('Ticket',vocab)
one_hot = tf.feature_column.embedding_column(cat, dimension=9)
feature_columns.append( one_hot )







In [ ]:
# 참고 DenseFeatures에 들어가는 데이터 출력해보기

ds.batch = ds.batch(32)

next(iter(ds_batch))[0]

feature_layer = tf.keras.layers.DenseFeatures( tf.feature_column.numeric_column('Fare') )
feature_layer(next(iter(ds_batch))[0])





<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[  7.25  ],
       [ 71.2833],
       [  7.925 ],
       [ 53.1   ],
       [  8.05  ],
       [  8.4583],
       [ 51.8625],
       [ 21.075 ],
       [ 11.1333],
       [ 30.0708],
       [ 16.7   ],
       [ 26.55  ],
       [  8.05  ],
       [ 31.275 ],
       [  7.8542],
       [ 16.    ],
       [ 29.125 ],
       [ 13.    ],
       [ 18.    ],
       [  7.225 ],
       [ 26.    ],
       [ 13.    ],
       [  8.0292],
       [ 35.5   ],
       [ 21.075 ],
       [ 31.3875],
       [  7.225 ],
       [263.    ],
       [  7.8792],
       [  7.8958],
       [ 27.7208],
       [146.5208]], dtype=float32)>

In [ ]:
model = tf.keras.Sequential([
                             
   tf.keras.layers.DenseFeatures(feature_columns), # Dict일때 꼭 제일먼저 들어가야하는 레이어                          
   tf.keras.layers.Dense(128, activation= 'relu'),
   tf.keras.layers.Dense(64,activation= 'relu'),
   tf.keras.layers.Dropout(0.2),
   tf.keras.layers.Dense(1, activation = 'sigmoid')


])

model.compile(optimizer = 'adam', loss ='binary_crossentropy', metrics =['acc'])

ds_batch= ds.batch(32)

model.fit(ds_batch, shuffle = True, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
28/28 [==============================] - 1s 3ms/step - loss: 0.6333 - acc: 0.6797
Epoch 2/20
28/28 [==============================] - 0s 2ms/step - loss: 0.4864 - acc: 0.8088
Epoch 3/20
28/28 [==============================] - 0s 3ms/step - loss: 0.4590 - acc: 0.7909
Epoch 4/20
28/28 [==============================] - 0s 2ms/step - loss: 0.4587 - acc: 0.8081
Epoch 5/20
28/28 [==============================] - 0s 2ms/step - loss: 0.4538 - acc: 0.8050
Epoch 6/20
28/28 [==============================] - 0s 2ms/step - loss: 0.4602 - acc: 0.8070
Epoch 7/20
28/28 [==============================] - 0s 2ms/step - loss: 0.4505 - acc: 0.8142
Epoch 8/20
28/28 [==============================] - 0s 2ms/step - loss: 0.4478 - acc: 0.8121
Epoch 9/20
28/28 [==============================] - 0s 3ms/step - loss: 0.4557 - acc: 0.8115
Epoch 10/20
28/28 [==============================] - 